# 1. Análisis de componentes principales



**Objetivo:** Transofrmar el espacio de características a un espacio con componentes principales.


**Información de las características**
Este dataset contiene imágenes que pertenecen all dataset de EuroSat. Hay 10 folders:
* 0 AnnualCrop
* 1 Forest
* 2 HerbaceousVegatation
* 3 Highway
* 4 Industrial
* 5 Pasture
* 6 PermanentCrop
* 7 Residential
* 8 River
* 9 SeaLake


**Número de instancias:** 27000

# 2. Autenticación a Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -U kaleido
!pip install plotly>=4.0.0
!wget https://github.com/plotly/orca/releases/download/v1.2.1/orca-1.2.1-x86_64.AppImage -O /usr/local/bin/orca
!chmod +x /usr/local/bin/orca
!apt-get install xvfb libgtk2.0-0 libgconf-2-4

# 3. Importando módulos

In [ ]:
import ____ as pd
from sklearn.decomposition import ____ #análisis de componentes principales
import plotly.express as px
import plotly.io as pio
import numpy as ___
import os
from sklearn.preprocessing import ____ #escalador

# 4. Lectura de archivo

In [ ]:
train_path = r'/content/drive/Shareddrives/Data Science para Geociencias/4. Selección de caracteristicas/Resultados/EUROSAT_TRAIN_FEAT.csv'
results_path = r'/content/drive/Shareddrives/Data Science para Geociencias/4. Selección de caracteristicas/Resultados'

In [ ]:
train_df = pd.read_csv(____)
train_df.head()

# 5. Escalamiento

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))
train_df.loc[:, train_df.columns != ___] = scaler.fit_transform(train_df.loc[:, train_df.columns != ___])

In [ ]:
_____.head()

# 6. PCA

#### a) PCA con 2 componentes

In [ ]:
pca = PCA(n_components=2)
pc = pca.fit_transform(train_df.iloc[:,:-1])
pca_df = pd.DataFrame(data = pc
             , columns = ['PC 1', 'PC 2'])
pca_df = pd.concat([pca_df, train_df[[____]]], axis = 1)

Varianza acumulada alcanzada

In [ ]:
print(np.sum(pca.explained_variance_ratio_))

Visualización del 2D

In [ ]:
fig = px.scatter(data_frame=____,
                    x='PC 1',
                    y='PC 2',
                    color='label',
                    color_continuous_scale=px.colors.sequential.Rainbow,
                    template='seaborn',
                    title='PCA 2D',
                    width=1000,
                    height=500)
# tight layout
fig.update_layout(margin=dict(l=0, r=0, b=0, t=25))
fig.update_traces(marker=dict(size=4),
                  selector=dict(mode='markers'))
pio.show(fig)
fig.write_html("pca_2.html")
fig.write_image("pca_2.png")

#### b) PCA con 3 componentes *principales* **texto en negrita**

In [ ]:
pca = PCA(n_components=3)
pc = pca.fit_transform(train_df.iloc[:,:-1])
pca_df = pd.DataFrame(data = pc
             , columns = ['PC 1', 'PC 2', 'PC 3'])
pca_df = pd.concat([_____, train_df[['label']]], axis = 1)

Poder explicativo

In [ ]:
print(np.sum(_____))

Visualización de PCA con 3 componentes

In [ ]:
fig = px.scatter_3d(data_frame=pca_df,
                    x='PC 1',
                    y='PC 2',
                    z='PC 3',
                    color='label',
                    color_continuous_scale=px.colors.sequential.Plasma,
                    template='seaborn',
                    title='PCA 3D EuroSat',
                    width=1300,
                    height=500)
# tight layout
fig.update_layout(margin=dict(l=0, r=0, b=0, t=25))
fig.update_traces(marker=dict(size=1),
                  selector=dict(mode='markers'))
pio.show(fig)
fig.write_html("pca_3.html")
fig.write_image("pca_3.png")

#### c) PCA con poder explicativo del 95%

In [ ]:
pca = PCA(0.95)
pc = pca.fit_transform(train_df.iloc[:,:-1])
df_pca_train = pd.DataFrame(data = ____,
                           columns=range(pc.shape[1]))
df_pca_train = pd.concat([____, train_df[['label']]], axis = 1)


Poder explicativo:

In [ ]:
print('Número de componentes principales: %s'%len(______))
print('Varianza acumulada con %s componentes: %s'%(len(pca.explained_variance_ratio_),np.sum(pca.explained_variance_ratio_)))

In [ ]:
_____.head()

Guradando el dataframe generado

In [ ]:
df_pca_train.to_csv(os.path.join(results_path,'EUROSAT_TRAIN_PCA.csv'))